In [1]:
!pip install langchain langchain_core langchain_groq langchain_community langchain langgraph

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 39.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.2/137.2 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.5/109.5 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.7/44.7 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.4 MB/s eta 0:00:00


In [2]:
import os
from typing import TypedDict, Annotated, List
from langgraph.graph import StateGraph, END
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables.graph import MermaidDrawMethod
from IPython.display import display, Image

In [3]:
class PlannerState(TypedDict):
  messages : Annotated[List[HumanMessage | AIMessage], "the messages in the conversation"]
  city: str
  interests: List[str]
  itinerary: str

In [5]:
from langchain_groq import ChatGroq
llm = ChatGroq(
    temperature = 0,
    groq_api_key = "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",
    model_name = "llama-3.3-70b-versatile"
)

In [6]:
itinerary_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful travel assistant. Create a day trip itinerary for {city} based on the user's interests: {interests}. Provide a brief, bulleted itinerary."),
    ("human", "Create an itinerary for my day trip."),
])


In [33]:
# Define PlannerState and message classes
PlannerState = Dict[str, Any]

class HumanMessage:
    def __init__(self, content: str):
        self.content = content

class AIMessage:
    def __init__(self, content: str):
        self.content = content

itinerary_prompt = "Create an itinerary for a trip to {city} with interests in {interests}."

# Step 1: Input city
def input_city(state: PlannerState) -> PlannerState:
    print("Please enter the city you want to visit for your day trip: ")
    user_message = input("Your Input: ")
    return {
        **state,
        "city": user_message,
        "messages": state['messages'] + [HumanMessage(content=user_message)]
    }

# Step 2: Input interests
def input_interest(state: PlannerState) -> PlannerState:
    print(f"Please enter your interests for the trip to {state['city']} (comma-separated): ")
    user_message = input("Your Input: ")
    return {
        **state,
        "interests": [interest.strip() for interest in user_message.split(",")],
        "messages": state['messages'] + [HumanMessage(content=user_message)]
    }

# Step 3: Input trip dates
def input_trip_dates(state: PlannerState) -> PlannerState:
    print(f"Please enter the starting date of your trip to {state['city']} (YYYY-MM-DD): ")
    start_date = input("Start Date: ")
    print(f"Please enter the ending date of your trip to {state['city']} (YYYY-MM-DD): ")
    end_date = input("End Date: ")
    return {
        **state,
        "start_date": start_date,
        "end_date": end_date,
        "messages": state['messages'] + [
            HumanMessage(content=f"Start Date: {start_date}"),
            HumanMessage(content=f"End Date: {end_date}")
        ]
    }

# Step 4: Input number of tourists by category
def input_tourist_count(state: PlannerState) -> PlannerState:
    print(f"Please enter the number of tourists for your trip to {state['city']} in the following categories:")
    men_count = int(input("Number of men: "))
    women_count = int(input("Number of women: "))
    children_count = int(input("Number of children: "))
    return {
        **state,
        "tourist_count": {
            "men": men_count,
            "women": women_count,
            "children": children_count
        },
        "messages": state['messages'] + [
            HumanMessage(content=f"Men: {men_count}, Women: {women_count}, Children: {children_count}")
        ]
    }

# Step 5: Input budget
def input_budget(state: PlannerState) -> PlannerState:
    print(f"Please enter your total budget for the trip to {state['city']} (in USD): ")
    user_message = input("Your Input: ")
    try:
        budget = float(user_message)
    except ValueError:
        print("Invalid input. Please enter a numeric value.")
        return input_budget(state)  # Retry on invalid input
    return {
        **state,
        "budget": budget,
        "messages": state['messages'] + [HumanMessage(content=user_message)]
    }

# Step 6: Create itinerary and include images
def create_itinerary(state: PlannerState) -> PlannerState:
    print(f"Creating an itinerary for {state['city']} based on interests: {', '.join(state['interests'])}")
    response = llm.invoke(itinerary_prompt.format(city=state['city'], interests=','.join(state['interests'])))
    print("\nFinal Itinerary:")
    print(response.content)

    # Generate images for each place in the itinerary (placeholder example)
    itinerary_places = response.content.split("\n")  # Assuming places are listed line by line
    images = {}
    for place in itinerary_places:
        if place.strip():
            print(f"Generating image for {place.strip()}...")
            images[place.strip()] = generate_image(place.strip())

    # Budget calculation (customized by category)
    cost_per_category = {"men": 100, "women": 90, "children": 50}  # Example costs
    total_budget = sum(
        count * cost_per_category[category]
        for category, count in state['tourist_count'].items()
    )

    print(f"Estimated Total Budget for the trip: ${total_budget}")

    return {
        **state,
        "messages": state['messages'] + [AIMessage(content=response.content)],
        "itinerary": response.content,
        "total_budget": total_budget,
        "images": images
    }

# Mock function to generate image for a place
def generate_image(place: str) -> str:
    # Here, you can integrate with an API like DALL-E or another image generator
    return f"Image for {place} (placeholder link or file path)"

# Main process to gather inputs and generate the itinerary
if __name__ == "__main__":
    state = {"messages": []}
    state = input_city(state)
    state = input_interest(state)
    state = input_trip_dates(state)
    state = input_tourist_count(state)
    state = input_budget(state)
    state = create_itinerary(state)


Please enter the city you want to visit for your day trip: 
Your Input: Ayodhya
Please enter your interests for the trip to Ayodhya (comma-separated): 
Your Input: Delhi
Please enter the starting date of your trip to Ayodhya (YYYY-MM-DD): 
Start Date: 2025-02-23
Please enter the ending date of your trip to Ayodhya (YYYY-MM-DD): 
End Date: 2025-02-26
Please enter the number of tourists for your trip to Ayodhya in the following categories:
Number of men: 2
Number of women: 2
Number of children: 0
Please enter your total budget for the trip to Ayodhya (in USD): 
Your Input: 10000
Creating an itinerary for Ayodhya based on interests: Delhi

Final Itinerary:
Ayodhya is a city in the Indian state of Uttar Pradesh, known for its rich history and cultural significance, particularly in Hinduism. If you're planning a trip to Ayodhya with interests in Delhi, here's a suggested 5-day itinerary:

Day 1: Delhi to Ayodhya

* Morning: Depart from Delhi by train or bus to Ayodhya (approx. 550 km, 9-10 

In [34]:
class StateGraph:
    def __init__(self, state_type):
        self.nodes = {}
        self.edges = {}
        self.entry_point = None

    def add_node(self, name: str, function: callable):
        self.nodes[name] = function

    def set_entry_point(self, name: str):
        self.entry_point = name

    def add_edge(self, from_node: str, to_node: str):
        if from_node not in self.edges:
            self.edges[from_node] = []
        self.edges[from_node].append(to_node)

    def compile(self):
        def app():
            state = {"messages": []}
            current_node = self.entry_point

            while current_node:
                state = self.nodes[current_node](state)
                next_nodes = self.edges.get(current_node, [])
                if not next_nodes:
                    break  # End the workflow
                current_node = next_nodes[0]  # Simple linear flow

        return app

# Update the workflow to include the new steps
workflow = StateGraph(PlannerState)

workflow.add_node("input_city", input_city)
workflow.add_node("input_interest", input_interest)
workflow.add_node("input_trip_dates", input_trip_dates)
workflow.add_node("input_tourist_count", input_tourist_count)
workflow.add_node("input_budget", input_budget)
workflow.add_node("create_itinerary", create_itinerary)

workflow.set_entry_point("input_city")

workflow.add_edge("input_city", "input_interest")
workflow.add_edge("input_interest", "input_trip_dates")
workflow.add_edge("input_trip_dates", "input_tourist_count")
workflow.add_edge("input_tourist_count", "input_budget")
workflow.add_edge("input_budget", "create_itinerary")

# Compiled workflow application
app = workflow.compile()

if __name__ == "__main__":
    app()


Please enter the city you want to visit for your day trip: 
Your Input: Ayodhya
Please enter your interests for the trip to Ayodhya (comma-separated): 
Your Input: Delhi
Please enter the starting date of your trip to Ayodhya (YYYY-MM-DD): 
Start Date: 2025-02-23
Please enter the ending date of your trip to Ayodhya (YYYY-MM-DD): 
End Date: 2025-02-26
Please enter the number of tourists for your trip to Ayodhya in the following categories:
Number of men: 2
Number of women: 2
Number of children: 0
Please enter your total budget for the trip to Ayodhya (in USD): 
Your Input: 10000
Creating an itinerary for Ayodhya based on interests: Delhi

Final Itinerary:
Ayodhya is a city in the Indian state of Uttar Pradesh, known for its rich history and cultural significance, particularly in Hinduism. If you're planning a trip to Ayodhya with interests in Delhi, here's a suggested 5-day itinerary:

Day 1: Delhi to Ayodhya

* Morning: Depart from Delhi by train or bus to Ayodhya (approx. 550 km, 9-10 

In [35]:
def travel_planner(user_request: str, budget: float):
    print(f"Initial Request: {user_request}\n")

    # Define the initial state of the user's request
    state = {
        "messages": [HumanMessage(content=user_request)],
        "city": "",  # To be filled based on user input
        "interests": [],  # List of user interests for personalized itinerary
        "start_date": "",  # To be filled with the trip start date
        "end_date": "",  # To be filled with the trip end date
        "tourist_count": {},  # Dictionary to store tourist count by category
        "itinerary": "",  # To be filled with the generated itinerary
        "budget": budget,  # User-defined budget
        "images": {},  # To store generated images for places in the itinerary
    }

    # Workflow steps
    state = input_city(state)
    state = input_interest(state)
    state = input_trip_dates(state)
    state = input_tourist_count(state)
    state = create_itinerary(state)

    # Final output
    print("\n--- Final Trip Details ---")
    print(f"City: {state['city']}")
    print(f"Interests: {', '.join(state['interests'])}")
    print(f"Trip Dates: {state['start_date']} to {state['end_date']}")
    print(f"Tourist Count: {state['tourist_count']}")
    print(f"Budget: ${state['budget']}")
    print(f"Itinerary:\n{state['itinerary']}")
    print("\nImages of Places in Itinerary:")
    for place, image in state["images"].items():
        print(f"{place}: {image}")

    return state





In [36]:
if __name__ == "__main__":
    user_request = "I want to plan a day trip"
    budget = 1000.0  # Example budget value

    # Run the travel planner with the user request and budget
    final_state = travel_planner(user_request, budget)

    # Print the final state for debugging or further processing
    print("\n--- Final State ---")
    for key, value in final_state.items():
        if key == "messages":  # Print messages as a list of their contents
            print(f"{key}: {[msg.content for msg in value]}")
        else:
            print(f"{key}: {value}")


Initial Request: I want to plan a day trip

Please enter the city you want to visit for your day trip: 
Your Input: Ayodhya
Please enter your interests for the trip to Ayodhya (comma-separated): 
Your Input: Delhi
Please enter the starting date of your trip to Ayodhya (YYYY-MM-DD): 
Start Date: 2025-02-23
Please enter the ending date of your trip to Ayodhya (YYYY-MM-DD): 
End Date: 2025-02-26
Please enter the number of tourists for your trip to Ayodhya in the following categories:
Number of men: 2
Number of women: 2
Number of children: 0
Creating an itinerary for Ayodhya based on interests: Delhi

Final Itinerary:
Ayodhya is a city in the Indian state of Uttar Pradesh, known for its rich history and cultural significance, particularly in Hinduism. If you're planning a trip to Ayodhya with interests in Delhi, here's a suggested 5-day itinerary:

Day 1: Delhi to Ayodhya

* Morning: Depart from Delhi by train or bus to Ayodhya (approx. 550 km, 9-10 hours journey)
* Afternoon: Check-in to 

In [37]:

!pip install gradio

In [38]:
import gradio as gr
from typing import TypedDict, Annotated, List
from datetime import datetime
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq

class PlannerState(TypedDict):
    messages: Annotated[List[HumanMessage | AIMessage], "The messages in the conversation"]
    city: str
    interests: List[str]
    itinerary: str
    start_date: str
    end_date: str
    num_men: int
    num_women: int
    num_others: int
    budget_men: float
    budget_women: float
    budget_others: float
    total_budget: float
    hotel_recommendations: str
    food_recommendations: str

# Define the LLM
llm = ChatGroq(
    temperature=0,
    groq_api_key="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",
    model_name="llama-3.3-70b-versatile"
)

# Define the itinerary prompt
itinerary_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful travel assistant. Create a day trip itinerary for {city} based on the user's interests: {interests}, dates: {start_date} to {end_date}, total members: {total_members}, and budget: {total_budget}. Provide a brief, bulleted itinerary and recommend hotels and food areas."),
    ("human", "Create an itinerary for my trip."),
])

# Input processing functions
def input_dates(start_date: str, end_date: str, state: PlannerState) -> PlannerState:
    return {
        **state,
        "start_date": start_date,
        "end_date": end_date,
        "messages": state["messages"] + [HumanMessage(content=f"Trip from {start_date} to {end_date}")],
    }

def input_budget_details(num_men: int, num_women: int, num_others: int, budget_men: float, budget_women: float, budget_others: float, state: PlannerState) -> PlannerState:
    total_budget = (num_men * budget_men) + (num_women * budget_women) + (num_others * budget_others)
    return {
        **state,
        "num_men": num_men,
        "num_women": num_women,
        "num_others": num_others,
        "budget_men": budget_men,
        "budget_women": budget_women,
        "budget_others": budget_others,
        "total_budget": total_budget,
        "messages": state["messages"] + [
            HumanMessage(content=f"Men: {num_men}, Women: {num_women}, Others: {num_others}, Total Budget: {total_budget}")
        ],
    }

def create_itinerary(state: PlannerState) -> str:
    try:
        response = llm.invoke(itinerary_prompt.format_messages(
            city=state["city"],
            interests=", ".join(state["interests"]),
            start_date=state["start_date"],
            end_date=state["end_date"],
            total_members=state["num_men"] + state["num_women"] + state["num_others"],
            total_budget=state["total_budget"],
        ))
        state["itinerary"] = response.content
        state["messages"] += [AIMessage(content=response.content)]
        return response.content
    except Exception as e:
        return f"An error occurred while generating the itinerary: {str(e)}"

# Define the Gradio application
def travel_planner(
    city: str,
    interests: str,
    start_date: str,
    end_date: str,
    num_men: int,
    num_women: int,
    num_others: int,
    budget_men: float,
    budget_women: float,
    budget_others: float
):
    # Initialize state
    state = {
        "messages": [],
        "city": city,
        "interests": [interest.strip() for interest in interests.split(",")],
        "itinerary": "",
        "start_date": start_date,
        "end_date": end_date,
        "num_men": num_men,
        "num_women": num_women,
        "num_others": num_others,
        "budget_men": budget_men,
        "budget_women": budget_women,
        "budget_others": budget_others,
        "total_budget": 0.0,
        "hotel_recommendations": "",
        "food_recommendations": "",
    }

    # Process budget details and calculate total budget
    state = input_budget_details(num_men, num_women, num_others, budget_men, budget_women, budget_others, state)

    # Process dates
    state = input_dates(start_date, end_date, state)

    # Generate the itinerary
    itinerary = create_itinerary(state)

    return f"{itinerary}\n\nTotal Budget: ${state['total_budget']:.2f}"

# Build the Gradio interface
interface = gr.Interface(
    fn=travel_planner,
    inputs=[
        gr.Textbox(label="Enter the city for your trip", placeholder="e.g., Paris"),
        gr.Textbox(label="Enter your interests (comma-separated)", placeholder="e.g., museums, food, art"),
        gr.Textbox(label="Trip Start Date (YYYY-MM-DD)", placeholder="e.g., 2025-01-15"),
        gr.Textbox(label="Trip End Date (YYYY-MM-DD)", placeholder="e.g., 2025-01-20"),
        gr.Number(label="Number of Men", value=1, precision=0),
        gr.Number(label="Number of Women", value=1, precision=0),
        gr.Number(label="Number of Others", value=0, precision=0),
        gr.Number(label="Budget per Man (USD)", value=100.0, precision=2),
        gr.Number(label="Budget per Woman (USD)", value=100.0, precision=2),
        gr.Number(label="Budget per Other (USD)", value=100.0, precision=2),
    ],
    outputs=gr.Textbox(label="Generated Itinerary"),
    title="Travel Itinerary Planner",
    description=(
        "Enter details about your trip, including the city, interests, trip dates, "
        "number of people (men, women, others), and individual budgets to generate a "
        "personalized itinerary with hotel and food recommendations."
    ),
)

# Launch the Gradio application
if __name__ == "__main__":
    interface.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c0f66d32bffb421872.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
